In [9]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split


In [10]:
# FINAL DATA WIHTOUT LOCATIONID COLUMNS

# In case kernel has crashed again, load the data here so you can skip all previous cells
FINAL_final_data = pd.read_parquet('data/FINAL_final_data_cleaned_scaled.parquet')
FINAL_final_data.to_csv()

# Again since kernel sometimes crashes
all_columns = FINAL_final_data.columns.tolist()
categorical_columns = ['RatecodeID', 'payment_type'] 
not_hot_columns = [col for col in all_columns if not any(cat_col in col for cat_col in categorical_columns)]

FINAL_final_data.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,extra,tip_amount,tolls_amount,RatecodeID_1.0,RatecodeID_2.0,...,RatecodeID_99.0,payment_type_1,payment_type_2,payment_type_3,payment_type_4,trip_duration,pickup_hour,pickup_day,peak_times,weekend
0,2024-12-01 00:12:27,2024-12-01 00:31:12,0.125,0.460595,0.391783,0.705882,0.236,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.214286,0.0,1.000000,0.0,1.0
1,2024-11-30 23:56:04,2024-12-01 00:28:15,0.125,0.359604,0.384371,0.117647,0.423,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.376459,1.0,0.833333,0.0,1.0
2,2024-12-01 00:18:16,2024-12-01 00:33:16,0.125,0.238320,0.273189,0.117647,0.250,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.255936,0.0,1.000000,0.0,1.0
3,2024-12-01 00:56:13,2024-12-01 01:18:25,0.125,0.202926,0.206480,0.411765,0.255,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.182093,0.0,1.000000,0.0,1.0
4,2024-12-01 00:21:17,2024-12-01 00:37:22,0.125,0.361491,0.362135,0.117647,0.402,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.303823,0.0,1.000000,0.0,1.0


In [14]:
# Lastly we set create two new dataframes for the features and target variable
y = FINAL_final_data['fare_amount']
X = FINAL_final_data.drop(columns=['fare_amount']) 

In [15]:
# Creating train test split
X_train, X_test, y_train, y_test = train_test_split(y, X, test_size=0.2, random_state=42)

In [16]:
# List of columns to drop
columns_to_drop = [
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'passenger_count',
    'tip_amount',
    'payment_type_1',
    'payment_type_2',
    'payment_type_3',
    'payment_type_4',
    'pickup_hour',
    'pickup_day'
]

# Drop the columns from the DataFrame
FINAL_final_data = FINAL_final_data.drop(columns=columns_to_drop, errors='ignore')

# Display the updated DataFrame
FINAL_final_data.head()


,trip_distance,fare_amount,extra,tolls_amount,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_6.0,RatecodeID_99.0,trip_duration,peak_times,weekend
0,0.460595,0.391783,0.705882,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.214286,0.0,1.0
1,0.359604,0.384371,0.117647,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.376459,0.0,1.0
2,0.238320,0.273189,0.117647,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.255936,0.0,1.0
3,0.202926,0.206480,0.411765,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.182093,0.0,1.0
4,0.361491,0.362135,0.117647,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.303823,0.0,1.0
